In [1]:
import os 
import sys
sys.path.append(os.path.join(os.getcwd(), '..'))
import pytorch_lightning as pl
from model.model_interface import LLM
from dataset.addition import Addition
import torch.utils.data as tud
from lightning.pytorch.loggers import TensorBoardLogger
import os 
import random


生成训练测试集

In [44]:
ub = 3
data_size = 200000
test_size = 0.6
lines = set()

# for i in range(10**ub):
#     for j in range(10**ub-i):
#         i = str(i).zfill(ub)
#         lines.add(f'{i}+{j}={i+j})
while len(lines) < data_size:
    l1 = random.randint(1, ub)
    l2 = random.randint(1, ub)
    a = random.randint(10**(l1-1), 10**l1-1)
    b_ub = 10**(ub) - a - 1
    b = random.randint(min(10**(l2-1), b_ub), min(10**l2, b_ub))
    c = a+b
    a = str(a).zfill(ub)
    b = str(b).zfill(ub)
    c = str(c).zfill(ub)
    lines.add(f'{a}+{b}={c}\n')

lines = list(lines)
random.shuffle(lines)
with open(f'data/train_{ub}_zfill.txt','w') as f:
    f.writelines(lines[int(len(lines)*test_size):])
with open(f'data/test_{ub}_zfill.txt','w') as f:
    f.writelines(lines[:int(len(lines)*test_size)])

训练配置

In [6]:
# LLM Config
llm_config = {
    "model_name": "gpt2math_small",
    "optimizer": "adam_w",
    "weight_decay": 1.0,
    "lr": 1e-4,
}

# Dataset config
dl_config = {
    "batch_size": 1024,
    "num_workers": 0,
}
data_dir= "data"
train_file= "train_2_zfill.txt"
test_file= "test_2_zfill.txt"

# Trainer config
trainer_config = {
    "precision" : "16-mixed",
    "accelerator" : "auto",
    "devices" : 1,
    "max_epochs": 500,
}

初始化模型，加载数据集，训练器

In [7]:
mt = LLM(**llm_config)
train_dl = tud.DataLoader(Addition(f"{data_dir}/{train_file}", mt.tokenizer), shuffle=True, **dl_config )
test_dl = tud.DataLoader(Addition(f"{data_dir}/{test_file}", mt.tokenizer), shuffle=False, **dl_config)
# default logger used by trainer (if tensorboard is installed)
logger = TensorBoardLogger(save_dir=os.getcwd(), version=1, name="lightning_logs")
trainer = pl.Trainer(**trainer_config, logger=logger)

Using 16bit Automatic Mixed Precision (AMP)
GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


开始训练！

In [8]:
trainer.fit(mt,train_dl,test_dl,)

You are using a CUDA device ('NVIDIA A100-SXM4-80GB') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]

  | Name  | Type            | Params
------------------------------------------
0 | model | GPT2LMHeadModel | 529 K 
------------------------------------------
529 K     Trainable params
0         Non-trainable params
529 K     Total params
2.119     Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

/nvme/yangyuchen1/miniconda3/envs/hug42/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/nvme/yangyuchen1/miniconda3/envs/hug42/lib/python3.8/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 128 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/nvme/yangyuchen1/miniconda3/envs/hug42/lib/python3.8/site-packages/pytorch_lightning/loops/fit_loop.py:280: PossibleUserWarning: The number of training batches (2)

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/nvme/yangyuchen1/miniconda3/envs/hug42/lib/python3.8/site-packages/pytorch_lightning/trainer/call.py:54: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


测试

In [49]:
test_res = trainer.test(mt,test_dl,)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0,1,2,3,4,5,6,7]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│          val_acc          │    0.9833333492279053     │
│         val_loss          │    0.12819461524486542    │
└───────────────────────────┴───────────────────────────┘